In [1]:
!pip install requests

In [2]:
import requests

In [3]:
import requests
import pandas as pd

In [ ]:
url = "https://api.sportradar.com/tennis/trial/v3/en/competitions.json?api_key=z1nYqGtU6IeGZbsoBQFYJjHJ3T3qYAmZd3kGn7BC"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

In [ ]:
data = response.json()

data['competitions'][0]

In [8]:
ab = []
for comp in data.get('competitions', []):
    ab.append({
        'competition_id': comp.get('id'),
        'competition_name': comp.get('name'),
        'type': comp.get('type'),
        'gender': comp.get('gender'),
        'parent_id': comp.get('parent_id', None),
        'cat_id': comp.get('category', {}).get('id', None),
        'category_name': comp.get('category', {}).get('name', None)
    })





In [ ]:
import pandas as pd
pd.DataFrame(ab)

In [ ]:
!pip install mysql-connector-python


In [11]:
import mysql.connector

# Establishing the connection to the local MySQL server
connection = mysql.connector.connect(
    host="localhost",         # MySQL server host (localhost for local machine)
    user="root",     # Your MySQL username
    password="Umaneethi@123", # Your MySQL password
    database="sportsradar"  # Database name you want to connect to
)

In [12]:
cursor = connection.cursor()


In [13]:
import mysql.connector
import pandas as pd

In [18]:


cursor = connection.cursor()

# Create tables
cursor.execute("""
CREATE TABLE IF NOT EXISTS Competition_table (
    competition_id INT ,
    competition_name VARCHAR(255),
    type VARCHAR(50),
    gender VARCHAR(50),
    parent_id VARCHAR(100),
    category_id VARCHAR(255) 
);
""")

cursor.execute("""
CREATE TABLE IF NOT EXISTS Category_table (
    category_id VARCHAR(255) ,
    category_name VARCHAR(255)
);
""")


In [ ]:
# Assuming 'ab' is a list of dictionaries
for row in ab:
    cursor.execute("""
    INSERT INTO Category_table (category_id, category_name)
    VALUES (%s, %s);
    """, (row['cat_id'], row['category_name']))

for row in ab:
    cursor.execute("""
    INSERT IGNORE INTO Competition_table (competition_id, competition_name, type, gender,parent_id,category_id)
    VALUES (%s, %s, %s, %s, %s, %s);
    """, (row['competition_id'], row['competition_name'], row['parent_id'], row['type'], row['gender'], row['cat_id']))

# Commit changes and close connection
connection.commit()
cursor.close()
connection.close()

print("Data has been inserted into MySQL!")


Data has been inserted into MySQL!


In [1]:
import mysql.connector
connection = mysql.connector.connect(
    host="localhost",         
    user="root",     
    password="Umaneethi@123",
    database="sportsradar"
)
cursor = connection.cursor()

In [2]:
!pip install tabulate

In [3]:
from tabulate import tabulate

In [4]:
# 1)List all competitions along with their category name
 
cursor.execute(
    '''SELECT b.competition_name,a.category_name FROM Category_table a 
    JOIN Competition_table b on a.category_id = b.category_id
    Limit 20
    '''             
)
results = cursor.fetchall()
headers = ["Competition Name", "Category Name"]
table = tabulate(results, headers, tablefmt="pipe")  #Grid ,Box ,Pipe, Fancy_grid
print(table)

| Competition Name                     | Category Name   |
|:-------------------------------------|:----------------|
| Hopman Cup                           | Hopman Cup      |
| ATP San Diego, USA Men Doubles       | ATP             |
| ATP San Diego, USA Men Singles       | ATP             |
| ATP Belgrade 2, Serbia Men Doubles   | ATP             |
| ATP Belgrade 2, Serbia Men Singles   | ATP             |
| ATP Parma, Italy Men Doubles         | ATP             |
| ATP Parma, Italy Men Singles         | ATP             |
| ATP Cagliari, Italy Men Doubles      | ATP             |
| ATP Cagliari, Italy Men Singles      | ATP             |
| ATP Marbella, Spain Men Doubles      | ATP             |
| ATP Marbella, Spain Men Singles      | ATP             |
| ATP Singapore, Singapore Men Doubles | ATP             |
| ATP Singapore, Singapore Men Singles | ATP             |
| ATP Melbourne, Australia Men Singles | ATP             |
| ATP Melbourne, Australia Men Doubles | ATP            

In [5]:
# 2)Count the number of competitions in each category

cursor.execute(
    '''SELECT a.category_name, COUNT(b.competition_id) AS competition_count 
       FROM Category_table a 
       LEFT JOIN Competition_table b ON a.category_id = b.category_id
       GROUP BY a.category_name
       Limit 20;
    '''             
)
results = cursor.fetchall()
headers = ["Competition Name", "Category Name"]
table = tabulate(results, headers, tablefmt="fancy_grid")  #Grid ,Box ,Pipe, Fancy_grid
print(table)

╒═════════════════════╤═════════════════╕
│ Competition Name    │   Category Name │
╞═════════════════════╪═════════════════╡
│ ATP                 │           49284 │
├─────────────────────┼─────────────────┤
│ Challenger          │          736164 │
├─────────────────────┼─────────────────┤
│ Davis Cup           │               1 │
├─────────────────────┼─────────────────┤
│ Exhibition          │             841 │
├─────────────────────┼─────────────────┤
│ Federation Cup      │               1 │
├─────────────────────┼─────────────────┤
│ Hopman Cup          │               1 │
├─────────────────────┼─────────────────┤
│ IPTL                │               1 │
├─────────────────────┼─────────────────┤
│ ITF Men             │         4831204 │
├─────────────────────┼─────────────────┤
│ ITF Women           │         4129024 │
├─────────────────────┼─────────────────┤
│ Juniors             │             256 │
├─────────────────────┼─────────────────┤
│ Legends             │           

In [7]:
# 3)Find all competitions of type 'doubles'
cursor.execute(
    '''SELECT competition_id ,competition_name ,gender FROM competition_table
       where gender = 'doubles'
       Limit 50;
    '''             
)
results = cursor.fetchall()
headers = ["CN_ID","Competition Name","Gender",]
table = tabulate(results, headers, tablefmt="fancy_grid")  #Grid ,Box ,Pipe, Fancy_grid
print(table)

╒═════════╤════════════════════════════════════════════════════╤══════════╕
│   CN_ID │ Competition Name                                   │ Gender   │
╞═════════╪════════════════════════════════════════════════════╪══════════╡
│       0 │ Wimbledon Men Doubles                              │ doubles  │
├─────────┼────────────────────────────────────────────────────┼──────────┤
│       0 │ Wimbledon Women Doubles                            │ doubles  │
├─────────┼────────────────────────────────────────────────────┼──────────┤
│       0 │ Australian Open Men Doubles                        │ doubles  │
├─────────┼────────────────────────────────────────────────────┼──────────┤
│       0 │ Australian Open Women Doubles                      │ doubles  │
├─────────┼────────────────────────────────────────────────────┼──────────┤
│       0 │ French Open Men Doubles                            │ doubles  │
├─────────┼────────────────────────────────────────────────────┼──────────┤
│       0 │ 

In [8]:
# 4) Get competitions that belong to a specific category (e.g., ITF Men)

cursor.execute(
    '''SELECT b.competition_name ,a.category_name
       FROM Category_table a 
       JOIN Competition_table b ON a.category_id = b.category_id
       WHERE a.category_name = 'ITF Men'
       Limit 100;
    '''             
)
results = cursor.fetchall()
headers = ["Competition Name", "Category Name"]
table = tabulate(results, headers, tablefmt="fancy_grid")  #Grid ,Box ,Pipe, Fancy_grid
print(table)

╒═══════════════════════════════════╤═════════════════╕
│ Competition Name                  │ Category Name   │
╞═══════════════════════════════════╪═════════════════╡
│ Dominican Republic F2 Men Singles │ ITF Men         │
├───────────────────────────────────┼─────────────────┤
│ Dominican Republic F2 Men Singles │ ITF Men         │
├───────────────────────────────────┼─────────────────┤
│ Dominican Republic F2 Men Singles │ ITF Men         │
├───────────────────────────────────┼─────────────────┤
│ Dominican Republic F2 Men Singles │ ITF Men         │
├───────────────────────────────────┼─────────────────┤
│ Dominican Republic F2 Men Singles │ ITF Men         │
├───────────────────────────────────┼─────────────────┤
│ Dominican Republic F2 Men Singles │ ITF Men         │
├───────────────────────────────────┼─────────────────┤
│ Dominican Republic F2 Men Singles │ ITF Men         │
├───────────────────────────────────┼─────────────────┤
│ Dominican Republic F2 Men Singles │ ITF Men   

In [9]:
# 5)Identify parent competitions and their sub-competitions

cursor.execute(
    '''SELECT b.competition_name,a.category_name FROM Category_table a 
    JOIN Competition_table b on a.category_id = b.category_id
    Limit 20
    '''             
)
results = cursor.fetchall()
headers = ["Competition Name", "Category Name"]
table = tabulate(results, headers, tablefmt="fancy_grid")  #Grid ,Box ,Pipe, Fancy_grid
print(table)

╒══════════════════════════════════════╤═════════════════╕
│ Competition Name                     │ Category Name   │
╞══════════════════════════════════════╪═════════════════╡
│ Hopman Cup                           │ Hopman Cup      │
├──────────────────────────────────────┼─────────────────┤
│ ATP San Diego, USA Men Doubles       │ ATP             │
├──────────────────────────────────────┼─────────────────┤
│ ATP San Diego, USA Men Singles       │ ATP             │
├──────────────────────────────────────┼─────────────────┤
│ ATP Belgrade 2, Serbia Men Doubles   │ ATP             │
├──────────────────────────────────────┼─────────────────┤
│ ATP Belgrade 2, Serbia Men Singles   │ ATP             │
├──────────────────────────────────────┼─────────────────┤
│ ATP Parma, Italy Men Doubles         │ ATP             │
├──────────────────────────────────────┼─────────────────┤
│ ATP Parma, Italy Men Singles         │ ATP             │
├──────────────────────────────────────┼────────────────

In [10]:
# 6)Analyze the distribution of competition types by category

cursor.execute(
    '''
    WITH CompetitionData AS (
        SELECT b.competition_name, a.category_name
        FROM Category_table a
        JOIN Competition_table b ON a.category_id = b.category_id
    )
    SELECT competition_name, category_name
    FROM CompetitionData
    LIMIT 20
    '''
)
results = cursor.fetchall()
headers = ["Competition Name", "Category Name"]
table = tabulate(results, headers, tablefmt="fancy_grid")  # Grid, Box, Pipe, Fancy_grid
print(table)


╒══════════════════════════════════════╤═════════════════╕
│ Competition Name                     │ Category Name   │
╞══════════════════════════════════════╪═════════════════╡
│ Hopman Cup                           │ Hopman Cup      │
├──────────────────────────────────────┼─────────────────┤
│ ATP San Diego, USA Men Doubles       │ ATP             │
├──────────────────────────────────────┼─────────────────┤
│ ATP San Diego, USA Men Singles       │ ATP             │
├──────────────────────────────────────┼─────────────────┤
│ ATP Belgrade 2, Serbia Men Doubles   │ ATP             │
├──────────────────────────────────────┼─────────────────┤
│ ATP Belgrade 2, Serbia Men Singles   │ ATP             │
├──────────────────────────────────────┼─────────────────┤
│ ATP Parma, Italy Men Doubles         │ ATP             │
├──────────────────────────────────────┼─────────────────┤
│ ATP Parma, Italy Men Singles         │ ATP             │
├──────────────────────────────────────┼────────────────

In [11]:
# 7) List all competitions with no parent (top-level competitions)

cursor.execute(
    '''SELECT b.competition_name,a.category_name FROM Category_table a 
    JOIN Competition_table b on a.category_id = b.category_id
    Limit 20
    '''             
)
results = cursor.fetchall()
headers = ["Competition Name", "Category Name"]
table = tabulate(results, headers, tablefmt="fancy_grid")  #Grid ,Box ,Pipe, Fancy_grid
print(table)

╒══════════════════════════════════════╤═════════════════╕
│ Competition Name                     │ Category Name   │
╞══════════════════════════════════════╪═════════════════╡
│ Hopman Cup                           │ Hopman Cup      │
├──────────────────────────────────────┼─────────────────┤
│ ATP San Diego, USA Men Doubles       │ ATP             │
├──────────────────────────────────────┼─────────────────┤
│ ATP San Diego, USA Men Singles       │ ATP             │
├──────────────────────────────────────┼─────────────────┤
│ ATP Belgrade 2, Serbia Men Doubles   │ ATP             │
├──────────────────────────────────────┼─────────────────┤
│ ATP Belgrade 2, Serbia Men Singles   │ ATP             │
├──────────────────────────────────────┼─────────────────┤
│ ATP Parma, Italy Men Doubles         │ ATP             │
├──────────────────────────────────────┼─────────────────┤
│ ATP Parma, Italy Men Singles         │ ATP             │
├──────────────────────────────────────┼────────────────